In [1]:
print('ok')

ok


In [2]:
%pwd

'/Users/rahulkulkarni/Downloads/Aaryan/Medical-Chatbot/research'

In [3]:
# Going one directory back so as to be able to load data
import os
os.chdir("../")

In [4]:
%pwd

'/Users/rahulkulkarni/Downloads/Aaryan/Medical-Chatbot'

In [5]:
## Document Loader
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader

def load_documents(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents


documents = load_documents("data")
print("Document loaded : ",documents[:2])
print("Total number of documents loaded : ",len(documents))



/Users/rahulkulkarni/Downloads/Aaryan/Medical-Chatbot/venv_medical/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Document loaded :  [Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'data/Medical_book.pdf', 'total_pages': 637, 'page': 0, 'page_label': '1'}, page_content=''), Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'data/Medical_book.pdf', 'total_pages': 637, 'page': 1, 'page_label': '2'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION')]
Total number of documents loaded :  637


In [6]:
documents[0]

Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'data/Medical_book.pdf', 'total_pages': 637, 'page': 0, 'page_label': '1'}, page_content='')

In [7]:
# Filter the document content
from typing import List
from langchain.docstore.document import Document
        
def load_minimal_docs(documents: List[Document]) -> List[Document]:
    minimal_docs = []
    for doc in documents:
        src = doc.metadata.get("source")
        minimal_doc = Document(page_content=doc.page_content, metadata={"source": src})
        minimal_docs.append(minimal_doc)

    return minimal_docs

filtered_docs = load_minimal_docs(documents)
print(filtered_docs[:2])





[Document(metadata={'source': 'data/Medical_book.pdf'}, page_content=''), Document(metadata={'source': 'data/Medical_book.pdf'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION')]


In [8]:
## Text Splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
def text_split(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    docs = text_splitter.split_documents(documents)

    return docs

text_chunks = text_split(filtered_docs)
print(len(text_chunks))

5859


In [10]:
## Embedding model to generate embeddings of the text chunks
from langchain_huggingface import HuggingFaceEmbeddings

def download_embedding_model():
    """Download the embedding model from HuggingFace."""
    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")   # Has a 384-dimensional embedding space)
    return embedding_model

embedding = download_embedding_model()
print("Embedding model downloaded")


Embedding model downloaded


In [11]:
vectors = embedding.embed_query('Hello world')
print(len(vectors))

384


In [12]:
from dotenv import load_dotenv
import os
load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_API_KEY
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["OPEAI_API_KEY"] = OPENAI_API_KEY

In [13]:
## Create Vector Store

from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medical-chatbot"
existing_indexes = [idx["name"] for idx in pc.list_indexes()]
if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=384,  # Dimension of the embedding model
        metric="cosine",
        spec=ServerlessSpec(cloud='aws', region='us-east-1'),
    )

index = pc.Index(index_name)

In [14]:
# Load the records in pinecone
from langchain_pinecone import PineconeVectorStore
vectorStore = PineconeVectorStore.from_documents(
    documents = text_chunks,
    embedding = embedding,
    index_name = index_name
)

In [15]:
type(vectorStore)

langchain_pinecone.vectorstores.PineconeVectorStore

In [16]:
vectorStore.similarity_search("What is hypertension?", k=2)


[Document(id='4472433f-5bc2-4988-a5f1-1f334027adf5', metadata={'source': 'data/Medical_book.pdf'}, page_content='(BPH), a condition that affects men and is characterized\nby an enlarged prostate gland.\nHigh blood pressure\nHigh blood pressure puts a strain on the heart and\nthe arteries. Over time, hypertension can damage the\nblood vessels to the point of causing stroke, heart fail-\nure or kidney failure. People with high blood pressure\nmay also be at higher risk for heart attacks. Controlling\nhigh blood pressure makes these problems less likely.\nAlpha blockers help lower blood pressure by causing'),
 Document(id='18144051-c21a-4ffb-8889-7a202edb9c27', metadata={'source': 'data/Medical_book.pdf'}, page_content='(BPH), a condition that affects men and is characterized\nby an enlarged prostate gland.\nHigh blood pressure\nHigh blood pressure puts a strain on the heart and\nthe arteries. Over time, hypertension can damage the\nblood vessels to the point of causing stroke, heart fail

In [17]:
## Retriever

retriever = vectorStore.as_retriever(search_type='mmr', lambda_mult=0.5, k=3)

retriever_docs = retriever.invoke("What is hypertension?")
print(retriever_docs)

[Document(metadata={'source': 'data/Medical_book.pdf'}, page_content='(BPH), a condition that affects men and is characterized\nby an enlarged prostate gland.\nHigh blood pressure\nHigh blood pressure puts a strain on the heart and\nthe arteries. Over time, hypertension can damage the\nblood vessels to the point of causing stroke, heart fail-\nure or kidney failure. People with high blood pressure\nmay also be at higher risk for heart attacks. Controlling\nhigh blood pressure makes these problems less likely.\nAlpha blockers help lower blood pressure by causing'), Document(metadata={'source': 'data/Medical_book.pdf'}, page_content='Hypertension—Persistently high arterial blood pres-\nsure.\nNeurotransmitter —Substance released from neu-\nrons of the peripheral nervous system that travels\nacross the synaptic clefts (gaps) of other neurons to\nexcite or inhibit the target cell.\nPalpitation—Rapid, forceful, throbbing, or fluttering\nheartbeat.\nReceptor—A molecular structure in a cell o

In [18]:
def format_docs(docs):
    return "\n".join([d.page_content for d in docs])

docs = format_docs(retriever_docs)
print(docs)

(BPH), a condition that affects men and is characterized
by an enlarged prostate gland.
High blood pressure
High blood pressure puts a strain on the heart and
the arteries. Over time, hypertension can damage the
blood vessels to the point of causing stroke, heart fail-
ure or kidney failure. People with high blood pressure
may also be at higher risk for heart attacks. Controlling
high blood pressure makes these problems less likely.
Alpha blockers help lower blood pressure by causing
Hypertension—Persistently high arterial blood pres-
sure.
Neurotransmitter —Substance released from neu-
rons of the peripheral nervous system that travels
across the synaptic clefts (gaps) of other neurons to
excite or inhibit the target cell.
Palpitation—Rapid, forceful, throbbing, or fluttering
heartbeat.
Receptor—A molecular structure in a cell or on the
surface of a cell that allows binding of a specific
substance that causes a specific physiologic
response.
tain kidney disorders increase the producti

In [19]:
# ## LLM
# from langchain_openai import ChatOpenAI 

# llm = ChatOpenAI(model="gpt-4o-mini")

# # llm.invoke('What is hypertension')

In [20]:
## LLM 
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

model = HuggingFaceEndpoint(
    repo_id = "HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    temperature = 0.2,
    max_new_tokens=256,
    top_p = 0.9,
)
llm = ChatHuggingFace(llm=model)




In [21]:
from langchain_core.messages import SystemMessage, HumanMessage

res = llm.invoke([
    SystemMessage(content='You are a medical assistant'),
    HumanMessage(content='What is Acne?'),
])
print(res.content)

Acne is a common skin condition characterized by the appearance of inflamed or infected sebum (oil) glands in the skin, typically on the face, chest, and back. It is caused by a combination of factors such as hormonal changes, bacteria, and excess oil production. Pores in the skin become clogged with oil and dead skin cells, leading to the formation of pimples, whiteheads, blackheads, and cysts. Acne can range from mild to severe and can leave scars if not treated properly. It is most commonly seen in adolescents and young adults but can affect people of all ages. Acne can be treated with over-the-counter medications, prescription medications, and lifestyle changes such as maintaining good hygiene and avoiding harsh skincare products. Severe acne may require medical intervention from a dermatologist.


In [23]:
## Prompt Template
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an Medical assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "Context:\n{context}"  
)

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{input}"),
    
])


In [24]:
# from pydantic import BaseModel, Field
# from langchain.output_parsers import PydanticOutputParser

# class Medical(BaseModel):
#     question: str = Field(description="The medical question to be answered.")
#     answer: str = Field(description="The answer to the medical question.")

# parser = PydanticOutputParser(pydantic_object=Medical)


In [25]:
## Output Parser
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [26]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda

retriever_chain = RunnableParallel({
    'input' : RunnablePassthrough(),
    'context' : retriever | RunnableLambda(format_docs)
})

# prompt template as Runnable
prompt_runnable = RunnableLambda(
    lambda inputs: prompt_template.format_messages(
        input=inputs['input'], 
        context=inputs['context']
    )
)

main_chain = retriever_chain | prompt_template | llm | parser

# result = main_chain.invoke('What is hypertension)
# print(result.summary)





In [32]:
results = main_chain.invoke('What are the benefits of doing Acupressure?')
print(results)


Acupressure can provide quick, cost-free, and effective relief from symptoms such as hay fever, sore throat, and heartburn, as well as increase energy and feelings of well-being, and alleviate sexual dysfunction. It can also be used to reduce stress and provide relief from symptoms and pain for conditions like headaches, general aches and pains, colds and flu, allergies, menstrual cramps, sinus problems, sprains, and tennis elbow. However, it should not replace professional healthcare for chronic conditions and should not be applied to open wounds, scar tissue, blisters, or areas with swelling and inflammation. Acupressure massage by a therapist can be very effective for prevention and treatment of many health conditions.
